In [1]:
# Basic
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

# AGAMA
import agama
units = agama.setUnits(length=1, velocity=1, mass=1)
print(units.replace(', ','\n'))

# Custom
from ICwriter import empty_data_dict, write_gadget_hdf

Length unit: 1 Kpc
velocity unit: 1 km/s
time unit: 977.793 Myr
mass unit: 1 Msun
gravitational constant: 4.30092e-06


In [2]:
data = empty_data_dict(Ntypes=2)
for key in data.keys(): print(f"data['{key}']:", data[key])

data['Type1']: {'pos': '(N,3):float', 'vel': '(N,3):float', 'ID': '(N):int'}
data['Type2']: {'pos': '(N,3):float', 'vel': '(N,3):float', 'ID': '(N):int'}


The example data container above has two place holders for two separate particle types.  
Each particle type requires a set of positions and velocities, which are both arrays of shape (N,3).  
Additionally, the particle types require (unique) IDs.  

In [3]:
def Spherical_IC(Nbody=1_000_000, M=1.5e11, rs=15):
    """
    Generates a simple, single component, spherical satellite 
    following a Hernquist profile.
    
    
    :Input:
    M: float
        Mass of the Hernquist in solar masses.
    rs: float
        scale radius of the Hernquist in kpc.
    
    Inspired by Vasiliev+2019:
    https://arxiv.org/abs/2009.10726
    """
    M = 1.5e11  # Mass in solar masses
    Rs = 15 # scale radius in kpc
    rho_0 = M/(2*np.pi)
    
    # Hernquist potential
    pot  = agama.Potential(type='spheroid', 
                           densityNorm=rho_0,
                           gamma=1, beta=4, alpha=1)
    
    # Eddington DF
    df   = agama.DistributionFunction(type='QuasiSpherical', 
                                      density=pot, 
                                      potential=pot)  
    
    return agama.GalaxyModel(pot, df).sample(Nbody)

See ./Tango4three_examples.py for some other examples to generate IC from the [Vasiliev+2019](https://arxiv.org/abs/2009.10726) paper

In [4]:
# Create an empty data structure for a single particle type
data = empty_data_dict(Ntypes=1)

# Generate IC following a Hernquist
IC, massess = Spherical_IC(Nbody=100_000)

In [5]:
# Just a sanity check
print("Total mass: {0:e}".format(massess.sum()))

mass = np.unique(massess)
print('Mass per particle:', mass)

Total mass: 1.496345e+11
Mass per particle: [1496345.19885321]


In [6]:
pos, vel = np.hsplit(IC, 2)
Npart = pos.shape[0]
Npart

100000

In [7]:
# Check: both should have shape (Npart, 3)
pos.shape, vel.shape

((100000, 3), (100000, 3))

In [8]:
#Type1
data['Type1']['pos'] = pos
data['Type1']['vel'] = vel
data['Type1']['ID'] = np.arange(Npart)


write_gadget_hdf('./Hernquist_100k.hdf5', 
                 data, 
                 NumPart=[0, Npart], # first element is for Type0
                 Massarr=[0, mass])

# Example: two particle types

# Type2
# data['Type2']['pos'] = pos2
# data['Type2']['vel'] = vel2
# data['Type2']['ID'] = np.arange(Npart2)+Npart

# write_gadget_hdf('./Hernquist_100k_2.hdf5', 
#                  data, 
#                  NumPart=[0, Npart, Npart2], # first element is for Type0
#                  Massarr=[0, mass, mass2])